# Paula Oviedo & Nicolas Patalagua

In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
from pyspark.sql import functions as f
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df=spark.read.csv("s3://newyorkcitybusdata/mta_1712.csv",header=True, inferSchema=True)
df2=spark.read.csv("s3://newyorkcitybusdata/mta_1710.csv",header=True, inferSchema=True)
df3=spark.read.csv("s3://newyorkcitybusdata/mta_1708.csv",header=True, inferSchema=True)
df4=spark.read.csv("s3://newyorkcitybusdata/mta_1706.csv",header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df=df.union(df2.select("*"))
df=df.union(df3.select("*"))
df=df.union(df4.select("*"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df.count() #6461753

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26522430

In [40]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- RecordedAtTime: timestamp (nullable = true)
 |-- DirectionRef: string (nullable = true)
 |-- PublishedLineName: string (nullable = true)
 |-- OriginName: string (nullable = true)
 |-- OriginLat: string (nullable = true)
 |-- OriginLong: string (nullable = true)
 |-- DestinationName: string (nullable = true)
 |-- DestinationLat: string (nullable = true)
 |-- DestinationLong: string (nullable = true)
 |-- VehicleRef: string (nullable = true)
 |-- VehicleLocation.Latitude: double (nullable = true)
 |-- VehicleLocation.Longitude: double (nullable = true)
 |-- NextStopPointName: string (nullable = true)
 |-- ArrivalProximityText: string (nullable = true)
 |-- DistanceFromStop: string (nullable = true)
 |-- ExpectedArrivalTime: string (nullable = true)
 |-- ScheduledArrivalTime: string (nullable = true)

In [41]:
columns_to_drop = ['OriginLat', 'OriginLong', 'DestinationLat', 'DestinationLong', 'VehicleLocation.Latitude','VehicleLocation.Longitude']
df = df.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- RecordedAtTime: timestamp (nullable = true)
 |-- DirectionRef: string (nullable = true)
 |-- PublishedLineName: string (nullable = true)
 |-- OriginName: string (nullable = true)
 |-- DestinationName: string (nullable = true)
 |-- VehicleRef: string (nullable = true)
 |-- NextStopPointName: string (nullable = true)
 |-- ArrivalProximityText: string (nullable = true)
 |-- DistanceFromStop: string (nullable = true)
 |-- ExpectedArrivalTime: string (nullable = true)
 |-- ScheduledArrivalTime: string (nullable = true)

In [43]:
#Las 5 estaciones que mas  concurrencia 
df.groupBy('PublishedLineName','OriginName','DestinationName').count().orderBy("count",ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+--------------------+------+
|PublishedLineName|          OriginName|     DestinationName| count|
+-----------------+--------------------+--------------------+------+
|          M15-SBS|       E 126 ST/2 AV|SELECT BUS SERVIC...|172880|
|          Q44-SBS|  BOSTON RD/E 180 ST|SELECT BUS JAMAIC...|159056|
|          Q44-SBS|MERRICK BL/ARCHER AV|SELECT BUS BRONX ...|158493|
|               B6|LIVONIA AV/ASHFOR...|LTD BENSONHURST H...|130130|
|             Bx36|   W 179 ST/BROADWAY|SOUNDVIEW PUGSLEY AV|130023|
+-----------------+--------------------+--------------------+------+
only showing top 5 rows

In [44]:
#Conteo por dias delso trayectos que se realizaron
fecha=df.select(f.split("RecordedAtTime"," ").getItem(0).alias("fecha"))
mes=fecha.select(f.split("fecha","-").getItem(1).alias("mes"),f.split("fecha","-").getItem(2).alias("dia"))
resul=mes.groupBy("mes",'dia').count().orderBy("mes",ascending=True).orderBy("dia",ascending=True).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+------+
|mes|dia| count|
+---+---+------+
| 12| 01|244305|
| 08| 01|249962|
| 10| 01|140736|
| 06| 01|248900|
| 12| 02|166630|
| 08| 02|248474|
| 10| 02|252501|
| 06| 02|250948|
| 12| 03|140573|
| 10| 03|252985|
| 06| 03|174420|
| 08| 03|247250|
| 06| 04|142007|
| 08| 04|254194|
| 10| 04|247837|
| 12| 04|243511|
| 08| 05|174908|
| 12| 05|245423|
| 10| 05|251930|
| 06| 05|251307|
+---+---+------+
only showing top 20 rows

In [47]:
#Los 5 veicilos que mas trayectos hicieron en los dos meses 
df.groupBy('VehicleRef').count().orderBy("count",ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|VehicleRef|count|
+----------+-----+
| NYCT_5857|10830|
| NYCT_5852|10588|
| NYCT_6047|10551|
| NYCT_7362|10426|
| NYCT_6037|10396|
+----------+-----+
only showing top 5 rows

In [50]:
fecha=df.select('VehicleRef',f.split("RecordedAtTime"," ").getItem(0).alias("fecha"))
mes=fecha.select('VehicleRef',f.split("fecha","-").getItem(1).alias("mes"))
print("Veiculo con mas trayectos en Junio")
resul1=mes.select('VehicleRef','mes').where("mes='06'").groupBy("VehicleRef").count().orderBy("count",ascending=False).show(1)
print("Veiculo con mas trayectos en Agosto")
resul1=mes.select('VehicleRef','mes').where("mes='08'").groupBy("VehicleRef").count().orderBy("count",ascending=False).show(1)
print("Veiculo con mas trayectos en octubre")
resul1=mes.select('VehicleRef','mes').where("mes='10'").groupBy("VehicleRef").count().orderBy("count",ascending=False).show(1)
print("Veiculo con mas trayectos en Diciembre")
resul1=mes.select('VehicleRef','mes').where("mes='12'").groupBy("VehicleRef").count().orderBy("count",ascending=False).show(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Veiculo con mas trayectos en Junio
+----------+-----+
|VehicleRef|count|
+----------+-----+
| NYCT_5860| 2978|
+----------+-----+
only showing top 1 row

Veiculo con mas trayectos en Agosto
+----------+-----+
|VehicleRef|count|
+----------+-----+
| NYCT_5852| 3306|
+----------+-----+
only showing top 1 row

Veiculo con mas trayectos en octubre
+----------+-----+
|VehicleRef|count|
+----------+-----+
| NYCT_5845| 3498|
+----------+-----+
only showing top 1 row

Veiculo con mas trayectos en Diciembre
+----------+-----+
|VehicleRef|count|
+----------+-----+
| NYCT_4606| 2794|
+----------+-----+
only showing top 1 row

In [80]:
#Timpo promedio que se trarda entre  estaciones 

Time=df.select(f.avg(f.unix_timestamp("ExpectedArrivalTime")-f.unix_timestamp("RecordedAtTime"))).show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------+
|avg((unix_timestamp(ExpectedArrivalTime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(RecordedAtTime, yyyy-MM-dd HH:mm:ss)))|
+---------------------------------------------------------------------------------------------------------------------+
|                                                                                                    60.03478337859938|
+---------------------------------------------------------------------------------------------------------------------+